<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [145]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

# 1. Download and Explore Dataset

### Load and explore the data

In [146]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

Let's take a quick look at the data.

In [147]:
df.head(5)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


The geographical coordinates of each postal code

In [148]:
df1=pd.read_csv("Geospatial_Coordinates.csv")
df1.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two datasets:

In [149]:
dd=pd.merge(df,df1,left_on='Postal code',right_on='Postal Code',how='left')
dd.head(5)

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1A,Not assigned,NaN,NaN,NaN,NaN
1,M2A,Not assigned,NaN,NaN,NaN,NaN
2,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
3,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
4,M5A,Downtown Toronto,Regent Park / Harbourfront,M5A,43.654260,-79.360636


### Cleaning of the dataset:

In [150]:
d = dd[dd['Borough']!='Not assigned']

Check:

In [151]:
d['Postal Code'].isna().sum()

0

In [152]:
d = d.drop('Postal Code',axis=1)
d = d.rename(columns={'Postal code':'PostalCode'})

Quickly examine the resulting dataframe.

In [153]:
d.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
5,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [154]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(d['Borough'].unique()),
        len(d['Neighborhood'].unique())
    )
)

The dataframe has 10 boroughs and 98 neighborhoods.


In [155]:
d=d.sort_values(by='PostalCode')
d.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
9,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
18,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
27,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
36,M1G,Scarborough,Woburn,43.770992,-79.216917
45,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# 2.Explore and cluster the neighborhoods in Downtown Toronto

For illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Downtown Toronto. So let's slice the original dataframe and create a new dataframe of the Downtown Toronto data.

In [156]:
DT_data = d[d['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DT_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,St. James Town / Cabbagetown,43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Let's get the geographical coordinates of Downtown Toronto.

In [157]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="TR_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


As we did with all of Toronto City, let's visualizat Downtown Toronto the neighborhoods in it.

In [158]:
# create map of Downtown Toronto using latitude and longitude values
map_DT = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(DT_data['Latitude'], DT_data['Longitude'], DT_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DT)  
    
map_DT

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [159]:
CLIENT_ID = 'ZZBPAF0R0X2C2U0C0IHS55CD0J2JB5JAWN4OX5NL3EG435W5' # your Foursquare ID
CLIENT_SECRET = 'XD2XTKDO3ZEDBRYBEKKNA53NW4VYHDKRNW4U3GJPHV5XGUKF' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Your credentails:
CLIENT_ID: ZZBPAF0R0X2C2U0C0IHS55CD0J2JB5JAWN4OX5NL3EG435W5
CLIENT_SECRET:XD2XTKDO3ZEDBRYBEKKNA53NW4VYHDKRNW4U3GJPHV5XGUKF


## (1). Explore Neighborhoods in Toronto Downtown

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto Downtown

In [160]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results=requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                 'Neighborhood Latitude', 
                 'Neighborhood Longitude', 
                 'Venue', 
                 'Venue Latitude', 
                'Venue Longitude', 
                 'Venue Category']
   
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [161]:
DT_venues = getNearbyVenues(names=DT_data['Neighborhood'],
                                   latitudes=DT_data['Latitude'],
                                   longitudes=DT_data['Longitude']
                                  )

Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Queen's Park / Ontario Provincial Government


#### Let's check the size of the resulting dataframe

In [162]:
print(DT_venues.shape)
DT_venues.head(5)

(1259, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,St. James Town / Cabbagetown,43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


Let's check how many venues were returned for each neighborhood

In [163]:
DT_venues.groupby('Neighborhood').count().head(5)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,73,73,73,73,73,73
Christie,18,18,18,18,18,18
Church and Wellesley,77,77,77,77,77,77


## (2). Analyze Each Neighborhood

In [164]:
# one hot encoding
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DT_onehot['Neighborhood'] = DT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]

DT_onehot.head(5)

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [165]:
DT_onehot.shape

(1259, 207)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [166]:
DT_grouped = DT_onehot.groupby('Neighborhood').mean().reset_index()
DT_grouped.head(5)

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,0.0,0.017857,0.000000,0.0,0.000000,0.017857,0.000000,0.017857,0.035714,0.0,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.0,0.0,0.017857,0.0000,0.000000,0.000000,0.0,0.017857,0.0,0.000000,0.0,0.000000,0.000000,0.017857,0.035714,0.0,0.000000,0.000000,0.035714,0.000000,0.0,0.0,0.000000,0.035714,0.053571,0.0,0.0,0.0,0.0,0.0,0.017857,0.000000,0.017857,0.017857,0.017857,0.0,0.000000,0.0,0.017857,0.000000,0.017857,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.017857,0.0,0.0,0.0,0.000000,0.035714,0.000000,0.0,0.0,0.017857,0.0,0.000000,0.0,0.0,0.017857,0.017857,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.017857,0.017857,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.017857,0.0,0.0,0.000000,0.017857,0.017857,0.035714,0.017857,0.017857,0.0,0.017857,0.000000,0.0,0.0,0.0,0.017857,0.017857,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.

#### Let's confirm the new size

In [167]:
DT_grouped.shape

(19, 207)

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [168]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 20 venues for each neighborhood.

In [169]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DT_grouped['Neighborhood']

for ind in np.arange(DT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(3)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Café,Cheese Shop,Restaurant,Bakery,Farmers Market,Beer Bar,Italian Restaurant,Japanese Restaurant,Eastern European Restaurant,Bistro,Cosmetics Shop,Indian Restaurant,Breakfast Spot,Irish Pub,Hotel,Jazz Club,Beach
1,CN Tower / King and Spadina / Railway Lands / ...,Airport Lounge,Airport Service,Harbor / Marina,Boat or Ferry,Plane,Rental Car Location,Sculpture Garden,Coffee Shop,Bar,Airport Terminal,Airport Gate,Airport Food Court,Airport,Boutique,Electronics Store,Distribution Center,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Gym / Fitness Center,Salad Place,Ice Cream Shop,Thai Restaurant,Spa,Yoga Studio,Modern European Restaurant,Fried Chicken Joint,Chinese Restaurant,Office,Park,Miscellaneous Shop


<a id='item4'></a>

## (3). Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 7 clusters.

In [170]:
# set number of clusters
kclusters = 7

DT_grouped_clustering = DT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 6, 3, 1, 5, 5, 1, 5, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [171]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

DT_merged = DT_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DT_merged = DT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DT_merged.head(3) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Creperie,Cupcake Shop,Electronics Store,Cosmetics Shop,Concert Hall,Comic Shop
1,M4X,Downtown Toronto,St. James Town / Cabbagetown,43.667967,-79.367675,5,Coffee Shop,Restaurant,Park,Pub,Italian Restaurant,Pizza Place,Bakery,Café,Chinese Restaurant,Breakfast Spot,Playground,Butcher,Pharmacy,Pet Store,Caribbean Restaurant,Grocery Store,Beer Store,Market,Deli / Bodega,Liquor Store
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Pizza Place,Hotel,Gastropub,Burger Joint,Yoga Studio,Mediterranean Restaurant,Men's Store,Pub,Café,Juice Bar,Italian Restaurant,Martial Arts Dojo,Indian Restaurant,Ice Cream Shop,Mexican Restaurant


Finally, let's visualize the resulting clusters

In [172]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DT_merged['Latitude'], DT_merged['Longitude'], DT_merged['Neighborhood'], DT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## (4). Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [173]:
DT_merged.loc[DT_merged['Cluster Labels'] == 0,DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
12,Downtown Toronto,0,Café,Bookstore,Japanese Restaurant,Bar,Italian Restaurant,Bakery,Restaurant,Beer Bar,Beer Store,Moving Target,Sandwich Place,Pub,Dessert Shop,Chinese Restaurant,Comfort Food Restaurant,Noodle House,Nightclub,College Arts Building,Bank,Yoga Studio


#### Cluster 2

In [174]:
DT_merged.loc[DT_merged['Cluster Labels'] == 1, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Pizza Place,Hotel,Gastropub,Burger Joint,Yoga Studio,Mediterranean Restaurant,Men's Store,Pub,Café,Juice Bar,Italian Restaurant,Martial Arts Dojo,Indian Restaurant,Ice Cream Shop,Mexican Restaurant
4,Downtown Toronto,1,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Bubble Tea Shop,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Ramen Restaurant,Restaurant,Theater,Tea Room,Bookstore,Lingerie Store,Electronics Store,Diner,Park,Department Store,Gym / Fitness Center
5,Downtown Toronto,1,Café,Coffee Shop,Cocktail Bar,Italian Restaurant,Beer Bar,Restaurant,Hotel,American Restaurant,Cosmetics Shop,Seafood Restaurant,Bakery,Clothing Store,Lingerie Store,Department Store,Japanese Restaurant,Diner,Gym,Farmers Market,Park,Gastropub
6,Downtown Toronto,1,Coffee Shop,Seafood Restaurant,Cocktail Bar,Café,Cheese Shop,Restaurant,Bakery,Farmers Market,Beer Bar,Italian Restaurant,Japanese Restaurant,Eastern European Restaurant,Bistro,Cosmetics Shop,Indian Restaurant,Breakfast Spot,Irish Pub,Hotel,Jazz Club,Beach
13,Downtown Toronto,1,Café,Vietnamese Restaurant,Coffee Shop,Bar,Vegetarian / Vegan Restaurant,Mexican Restaurant,Grocery Store,Farmers Market,Chinese Restaurant,Dumpling Restaurant,Dessert Shop,Gaming Cafe,Filipino Restaurant,Hospital,Ramen Restaurant,Poutine Place,Bubble Tea Shop,Burger Joint,Pizza Place,Pharmacy
15,Downtown Toronto,1,Coffee Shop,Restaurant,Seafood Restaurant,Café,Italian Restaurant,Hotel,Beer Bar,Japanese Restaurant,Cheese Shop,Art Gallery,Breakfast Spot,Park,Gym,Bakery,Farmers Market,Cosmetics Shop,Creperie,Cocktail Bar,Church,Juice Bar


#### Cluster 3

In [175]:
DT_merged.loc[DT_merged['Cluster Labels'] == 2, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Downtown Toronto,2,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Creperie,Cupcake Shop,Electronics Store,Cosmetics Shop,Concert Hall,Comic Shop


#### Cluster 4

In [176]:
DT_merged.loc[DT_merged['Cluster Labels'] == 3, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
17,Downtown Toronto,3,Grocery Store,Café,Park,Baby Store,Nightclub,Coffee Shop,Restaurant,Gas Station,Italian Restaurant,Athletics & Sports,Diner,Candy Store,Dessert Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Airport


#### Cluster 4

In [177]:
DT_merged.loc[DT_merged['Cluster Labels'] == 4, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
14,Downtown Toronto,4,Airport Lounge,Airport Service,Harbor / Marina,Boat or Ferry,Plane,Rental Car Location,Sculpture Garden,Coffee Shop,Bar,Airport Terminal,Airport Gate,Airport Food Court,Airport,Boutique,Electronics Store,Distribution Center,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [178]:
DT_merged.loc[DT_merged['Cluster Labels'] == 5, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,Downtown Toronto,5,Coffee Shop,Restaurant,Park,Pub,Italian Restaurant,Pizza Place,Bakery,Café,Chinese Restaurant,Breakfast Spot,Playground,Butcher,Pharmacy,Pet Store,Caribbean Restaurant,Grocery Store,Beer Store,Market,Deli / Bodega,Liquor Store
3,Downtown Toronto,5,Coffee Shop,Pub,Park,Bakery,Café,Breakfast Spot,Mexican Restaurant,Chocolate Shop,Event Space,Spa,Hotel,Ice Cream Shop,Shoe Store,Farmers Market,Restaurant,Electronics Store,Bank,Distribution Center,Dessert Shop,Performing Arts Venue
8,Downtown Toronto,5,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Deli / Bodega,Hotel,Bakery,Cosmetics Shop,Concert Hall,Bar,Lounge,Asian Restaurant,Salad Place,Office,Bookstore,Breakfast Spot,Steakhouse,Pizza Place,Gym / Fitness Center
9,Downtown Toronto,5,Coffee Shop,Aquarium,Restaurant,Café,Hotel,Italian Restaurant,Scenic Lookout,Sporting Goods Shop,Brewery,Fried Chicken Joint,Park,Bar,Plaza,Pizza Place,History Museum,Sports Bar,Baseball Stadium,Music Venue,Bubble Tea Shop,Bistro
10,Downtown Toronto,5,Coffee Shop,Hotel,Café,Restaurant,Bar,Gastropub,Seafood Restaurant,American Restaurant,Italian Restaurant,Concert Hall,Salad Place,Bakery,Beer Bar,Asian Restaurant,Sporting Goods Shop,Steakhouse,Lounge,Japanese Restaurant,Tea Room,Deli / Bodega
11,Downtown Toronto,5,Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Deli / Bodega,Japanese Restaurant,Seafood Restaurant,Italian Restaurant,Gastropub,Thai Restaurant,Vegetarian / Vegan Restaurant,Bar,Beer Bar,Bakery,Burger Joint,Building,Park,Museum
16,Downtown Toronto,5,Coffee Shop,Café,Restaurant,Gym,Seafood Restaurant,Asian Restaurant,Japanese Restaurant,Steakhouse,American Restaurant,Hotel,Gastropub,Concert Hall,Bar,Tea Room,Bakery,Pizza Place,Deli / Bodega,Thai Restaurant,Beer Bar,Brazilian Restaurant


In [179]:
DT_merged.loc[DT_merged['Cluster Labels'] == 6, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
7,Downtown Toronto,6,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Gym / Fitness Center,Salad Place,Ice Cream Shop,Thai Restaurant,Spa,Yoga Studio,Modern European Restaurant,Fried Chicken Joint,Chinese Restaurant,Office,Park,Miscellaneous Shop
18,Downtown Toronto,6,Coffee Shop,Diner,Yoga Studio,Beer Bar,Distribution Center,Discount Store,Bar,Bank,Mexican Restaurant,Burrito Place,Burger Joint,Park,Hobby Shop,Arts & Crafts Store,Art Gallery,Sandwich Place,Fried Chicken Joint,Juice Bar,Theater,Creperie


### Thank you for reviewing!

This notebook was created by [Xiaoyu](https://www.linkedin.com/in/xiaoyu-guo-377021112/). 